In [1]:
import sys
import os
import numpy as np
import numpy.ma as ma
from glob import glob
from time import strptime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib as mplz
import matplotlib.pyplot as plt
import pandas as pd
import netCDF4
import datetime
from netCDF4 import num2date
import cftime
import statistics
import scipy.io as sio
import xarray as xr
import matplotlib.dates as mdates
import nc_time_axis 
from datetime import datetime as real_datetime 
import nc_time_axis
from math import pi
from numpy import cos, sin

In [2]:
dir_R18O = '/glade/p/cesm/community/cesmLME/CESM-CAM5-LME/ocn/proc/tseries/monthly/R18O/'

In [3]:
full_forcing1 = ["b.ie12.B1850CN.f19_g16.001"]
full_forcing2 = ["b.ie12.B1850C5CN.f19_g16.LME.002"]
full_forcing3 = ["b.ie12.B1850C5CN.f19_g16.LME.003" ]
ghg = ["b.ie12.B1850C5CN.f19_g16.LME.GHG.001"]
orb = ["b.ie12.B1850C5CN.f19_g16.LME.Orbital.001"]
sol = ["b.ie12.B1850C5CN.f19_g16.LME.SSI_VSK_L.001"]
vol1 = ["b.ie12.B1850C5CN.f19_g16.LME.VOLC_GRA.001"]
vol2 = ["b.ie12.B1850C5CN.f19_g16.LME.VOLC_GRA.002"]

## Read In Data

In [4]:
## Read in Coral Data
data = pd.read_csv('iso2k_clean.csv')

In [5]:
nc1 = netCDF4.Dataset('/glade/p/cesm/community/cesmLME/CESM-CAM5-LME/ocn/proc/tseries/monthly/R18O/b.ie12.B1850CN.f19_g16.001.pop.h.R18O.185001-200512.nc')

In [6]:
r18 = nc1.variables['R18O']

In [7]:
latvar = nc1.variables['TLAT']
lonvar = nc1.variables['TLONG']

In [8]:
## Get lon, lat, and record data from csv
paleo_lon = xr.DataArray(data['lon'].values)
paleo_lat = xr.DataArray(data['lat'].values)
test_record = xr.DataArray(data['record'].values)

In [9]:
paleo_long = np.asarray(paleo_lon)

In [10]:
test_lon = paleo_long.tolist()

In [11]:
paleo_lat = np.asarray(paleo_lat)

In [12]:
test_lat = paleo_lat.tolist()

##  Find closest point in a set of (lat,lon) points to specified point

In [13]:
#Find closest point in a set of (lat,lon) points to specified point
# latvar - 2D latitude variable from an open netCDF dataset
# lonvar - 2D longitude variable from an open netCDF dataset
# lat0,lon0 - query point
#Returns iy,ix such that the square of the tunnel distance between (latval[it,ix],lonval[iy,ix]) and (lat0,lon0)is minimum. 

In [14]:
def tunnel_fast(latvar,lonvar,lat0,lon0):
    rad_factor = pi/180.0 # for trignometry, need angles in radians
    # Read latitude and longitude from file into numpy arrays
    latvals = latvar[:] * rad_factor
    lonvals = lonvar[:] * rad_factor
    ny,nx = latvals.shape
    lat0_rad = lat0 * rad_factor
    lon0_rad = lon0 * rad_factor
    # Compute numpy arrays for all values, no loops
    clat,clon = cos(latvals),cos(lonvals)
    slat,slon = sin(latvals),sin(lonvals)
    delX = cos(lat0_rad)*cos(lon0_rad) - clat*clon
    delY = cos(lat0_rad)*sin(lon0_rad) - clat*slon
    delZ = sin(lat0_rad) - slat;
    dist_sq = delX**2 + delY**2 + delZ**2
    minindex_1d = dist_sq.argmin()  # 1D index of minimum element
    iy_min,ix_min = np.unravel_index(minindex_1d, latvals.shape)
    return iy_min,ix_min

In [15]:
new_lat = [] 
new_lon = []
for i in range(0, len(test_lat)):
        filtered_lat, filtered_lon= tunnel_fast(latvar, lonvar, test_lat[i],test_lon[i])
        new_lat.append(filtered_lat)
        new_lon.append(filtered_lon)

In [19]:
ens_sel=[]

for n in full_forcing1:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
full_forcing_post1 = full_df.pivot_table('R18O', ['time'], 'record')    

In [28]:
full_forcing_post3.to_csv('test_full_post3.csv')

In [17]:
full_forcing1_rename = full_forcing_post1.set_axis(test_record, axis=1, inplace=False)
full_forcing1_rename.to_csv('d18O_full_post1.csv')

ValueError: Length mismatch: Expected axis has 77 elements, new values have 82 elements

In [20]:
ens_sel=[]

for n in full_forcing2:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
full_forcing_post2 = full_df.pivot_table('R18O', ['time'], 'record')    

In [25]:
full_forcing2_rename = full_forcing_post2.set_axis(test_record, axis=1, inplace=False)
full_forcing2_rename.to_csv('d18O_full_post2.csv')

ValueError: Length mismatch: Expected axis has 77 elements, new values have 82 elements

In [21]:
ens_sel=[]

for n in full_forcing3:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
full_forcing_post3 = full_df.pivot_table('R18O', ['time'], 'record')    

In [2]:
full_forcing_post3

NameError: name 'full_forcing_post3' is not defined

In [29]:
full_forcing3_rename = full_forcing_post3.set_axis(test_record, axis=1, inplace=False)
full_forcing3_rename.to_csv('d18O_full_post3.csv')

ValueError: Length mismatch: Expected axis has 77 elements, new values have 82 elements

## GHG

In [17]:
ens_sel=[]

for n in ghg:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
ghg_post = full_df.pivot_table('R18O', ['time'], 'record')

In [18]:
ghg_post.to_csv('test_ghg_post.csv')

In [19]:
ghg_rename = ghg_post.set_axis(test_record, axis=1, inplace=False)
ghg_rename.to_csv('d18O_ghg_post.csv')

## Orbital

In [20]:
ens_sel=[]

for n in orb:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
orb_post = full_df.pivot_table('R18O', ['time'], 'record')

In [23]:
orb_rename = orb_post.set_axis(test_record, axis=1, inplace=False)
orb_rename.to_csv('d18O_orb_post.csv')

## Solar

In [24]:
ens_sel=[]

for n in sol:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
sol_post = full_df.pivot_table('R18O', ['time'], 'record')

In [29]:
sol_rename = sol_post.set_axis(test_record, axis=1, inplace=False)
sol_rename.to_csv('d18O_sol_post.csv')

## Volcanic

In [25]:
ens_sel=[]

for n in vol1:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
vol1_post = full_df.pivot_table('R18O', ['time'], 'record')

In [28]:
vol1_rename = vol1_post.set_axis(test_record, axis=1, inplace=False)
vol1_rename.to_csv('d18O_vol_post1.csv')

In [26]:
ens_sel=[]

for n in vol2:
    sel = []
    its = xr.open_dataset(dir_R18O + n + ".pop.h.R18O.185001-200512.nc")
    for i in range(0, len(test_lat)):
        filtered_its1=  its["R18O"].sel(nlat =new_lat[i],nlon=new_lon[i], z_t =500)
        filtered_itsd = ((filtered_its1)-1)*1000
        sel.append(filtered_itsd)
    test_mem = xr.concat(sel, dim= "record")
    ens_sel.append(test_mem)
    
test_ens=xr.concat(ens_sel,dim="members")
full_df = test_ens.to_dataframe()
vol2_post = full_df.pivot_table('R18O', ['time'], 'record')

In [27]:
vol2_rename = vol2_post.set_axis(test_record, axis=1, inplace=False)
vol2_rename.to_csv('d18O_vol2.csv')